In [1]:
import os
import sys

src_path = os.path.abspath('..')
print(src_path)
sys.path.append(src_path)

/home/zw12/MedLink/src


In [2]:
from utils import data_path, set_seed, create_directory

In [3]:
import csv
import json
from datetime import datetime
from collections import Counter, OrderedDict
import random
import pandas as pd
import numpy as np
from itertools import chain

In [4]:
set_seed(seed=42)

## patient info

In [5]:
patients = pd.read_csv(os.path.join(data_path, 'mimic3/raw/PATIENTS.csv.gz'))
print(patients.shape)
patients.head()

(46520, 8)


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [6]:
patients['DOB'] = pd.to_datetime(patients['DOB']).dt.date

In [7]:
patients = patients[['SUBJECT_ID', 'GENDER', 'DOB']]
patients

,SUBJECT_ID,GENDER,DOB
0,249,F,2075-03-13
1,250,F,2164-12-27
2,251,M,2090-03-15
3,252,M,2078-03-06
4,253,F,2089-11-26
...,...,...,...
46515,44089,M,2026-05-25
46516,44115,F,2124-07-27
46517,44123,F,2049-11-26
46518,44126,F,2076-07-25


## admission info

In [8]:
admissions = pd.read_csv(os.path.join(data_path, 'mimic3/raw/ADMISSIONS.csv.gz'))
print(admissions.shape)
admissions.head()

(58976, 19)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [9]:
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME']).dt.date
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME']).dt.date

In [10]:
admissions = admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 
                         'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',  
                         'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']]

## diagnosis code info

In [11]:
diagnosis_icd = pd.read_csv(os.path.join(data_path, 'mimic3/raw/DIAGNOSES_ICD.csv.gz'))
print(diagnosis_icd.shape)
diagnosis_icd.head()

(651047, 5)


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


ICD9 FULL

In [12]:
diagnosis_icd = diagnosis_icd.rename(columns={'ICD9_CODE': 'ICD9_FULL_CODE'})
diagnosis_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_FULL_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [13]:
# drop NaN code
print("# of rows with NaN code:", np.count_nonzero(diagnosis_icd['ICD9_FULL_CODE'].isnull()))
diagnosis_icd = diagnosis_icd.dropna(subset=['ICD9_FULL_CODE'])
print("Rows with NaN code are dropped! Shape:", diagnosis_icd.shape)

# of rows with NaN code: 47
Rows with NaN code are dropped! Shape: (651000, 5)


In [14]:
diagnosis_icd['ICD9_FULL_CODE'].nunique()

6984

ICD9 3-Digit

In [15]:
def convert_to_3digit_icd9(dxStr):
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3]
        else: return dxStr

In [16]:
diagnosis_icd['ICD9_3DIGIT_CODE'] = diagnosis_icd['ICD9_FULL_CODE'].apply(lambda x: convert_to_3digit_icd9(x))
diagnosis_icd

/tmp/ipykernel_1694070/3364398192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnosis_icd['ICD9_3DIGIT_CODE'] = diagnosis_icd['ICD9_FULL_CODE'].apply(lambda x: convert_to_3digit_icd9(x))


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_FULL_CODE,ICD9_3DIGIT_CODE
0,1297,109,172335,1.0,40301,403
1,1298,109,172335,2.0,486,486
2,1299,109,172335,3.0,58281,582
3,1300,109,172335,4.0,5855,585
4,1301,109,172335,5.0,4254,425
...,...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280,202
651043,639799,97503,188195,3.0,V5869,V58
651044,639800,97503,188195,4.0,V1279,V12
651045,639801,97503,188195,5.0,5275,527


In [17]:
diagnosis_icd['ICD9_3DIGIT_CODE'].nunique()

1070

CCS

In [18]:
icd9_to_ccs = {}
with open(os.path.join(data_path, 'resource/$dxref 2015.csv')) as f:
    lines = f.readlines()
    for line in lines[3:]:
        line = line.split(',')
        icd9_code = line[0].strip("'").strip()
        ccs_code = line[1].strip("'").strip()
        assert icd9_code not in icd9_to_ccs
        icd9_to_ccs[icd9_code] = ccs_code

In [19]:
len(icd9_to_ccs)

15072

In [20]:
len(set(diagnosis_icd['ICD9_FULL_CODE'].unique()).intersection(set(icd9_to_ccs)))

6984

In [21]:
diagnosis_icd['CCS_CODE'] = diagnosis_icd['ICD9_FULL_CODE'].apply(lambda x: icd9_to_ccs[x])
diagnosis_icd

/tmp/ipykernel_1694070/3566129038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnosis_icd['CCS_CODE'] = diagnosis_icd['ICD9_FULL_CODE'].apply(lambda x: icd9_to_ccs[x])


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE
0,1297,109,172335,1.0,40301,403,99
1,1298,109,172335,2.0,486,486,122
2,1299,109,172335,3.0,58281,582,156
3,1300,109,172335,4.0,5855,585,158
4,1301,109,172335,5.0,4254,425,97
...,...,...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280,202,38
651043,639799,97503,188195,3.0,V5869,V58,257
651044,639800,97503,188195,4.0,V1279,V12,155
651045,639801,97503,188195,5.0,5275,527,137


In [22]:
diagnosis_icd['CCS_CODE'].nunique()

281

Agg

In [23]:
diagnosis_icd = diagnosis_icd[['SUBJECT_ID', 'HADM_ID', 'ICD9_FULL_CODE', 'ICD9_3DIGIT_CODE', 'CCS_CODE']]

In [24]:
diagnosis_icd = diagnosis_icd.groupby(['SUBJECT_ID', 'HADM_ID']).agg({'ICD9_FULL_CODE': list, 'ICD9_3DIGIT_CODE': list, 'CCS_CODE': list}).reset_index()
diagnosis_icd

,SUBJECT_ID,HADM_ID,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE
0,2,163353,"[V3001, V053, V290]","[V30, V05, V29]","[218, 10, 256]"
1,3,145834,"[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4...","[038, 785, 584, 427, 410, 428, 682, 425, 263]","[2, 249, 157, 107, 100, 108, 197, 97, 52]"
2,4,185777,"[042, 1363, 7994, 2763, 7907, 5715, 04111, V09...","[042, 136, 799, 276, 790, 571, 041, V09, E931]","[5, 122, 52, 55, 2, 151, 3, 3, 2617]"
3,5,178980,"[V3000, V053, V290]","[V30, V05, V29]","[218, 10, 256]"
4,6,107064,"[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1...","[403, 444, 997, 276, 276, 285, 275, V15]","[99, 116, 238, 55, 55, 59, 58, 663]"
...,...,...,...,...,...
58924,99985,176670,"[0389, 51881, 48241, 4870, 78552, V4281, 99592...","[038, 518, 482, 487, 785, V42, 995, 244, 272, ...","[2, 131, 122, 123, 249, 259, 2, 48, 53, 59, 13..."
58925,99991,151118,"[56211, 0389, 5570, 5849, 99592, 56081, 78959,...","[562, 038, 557, 584, 995, 560, 789, 553, 788, ...","[146, 2, 114, 157, 2, 145, 151, 143, 163, 99, ..."
58926,99992,197084,"[9999, 56881, 5772, 2851, 5849, 5799, 72992, 5...","[999, 568, 577, 285, 584, 579, 729, 530, 401, ...","[238, 155, 152, 60, 157, 155, 211, 138, 98, 53..."
58927,99995,137810,"[4414, 42833, 99812, 2851, 4241, 25000, 99811,...","[441, 428, 998, 285, 424, 250, 998, 996, E879,...","[115, 108, 238, 60, 96, 49, 238, 237, 2616, 53..."


In [25]:
diagnosis_icd.ICD9_FULL_CODE = diagnosis_icd.ICD9_FULL_CODE.apply(lambda x: ' '.join(list(set(x))))
diagnosis_icd.ICD9_3DIGIT_CODE = diagnosis_icd.ICD9_3DIGIT_CODE.apply(lambda x: ' '.join(list(set(x))))
diagnosis_icd.CCS_CODE = diagnosis_icd.CCS_CODE.apply(lambda x: ' '.join(list(set(x))))

## merge

In [26]:
data = pd.merge(patients, admissions, how='inner', on='SUBJECT_ID')
data = pd.merge(data, diagnosis_icd, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
data

,SUBJECT_ID,GENDER,DOB,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE
0,249,F,2075-03-13,116935,2149-12-17,2149-12-31,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,42841 41402 7921 99812 51882 41071 53081 4589 ...,410 427 530 466 458 584 998 518 250 285 414 42...,106 125 157 128 100 131 238 49 108 117 237 138...
1,249,F,2075-03-13,149546,2155-02-03,2155-02-14,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,CATHOLIC,DIVORCED,WHITE,43411 41402 5859 49320 78451 41071 4280 53081 ...,569 410 427 272 530 584 285 434 414 493 342 42...,106 60 157 53 100 128 99 49 108 95 48 158 82 2...
2,249,F,2075-03-13,158975,2156-04-27,2156-05-14,EMERGENCY,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,ENGL,CATHOLIC,DIVORCED,WHITE,E8788 4280 25000 1982 5853 59971 V1254 5601 45...,153 599 263 E878 428 560 250 244 403 427 584 9...,106 259 157 52 101 2616 59 128 99 49 117 138 4...
3,250,F,2164-12-27,124271,2188-11-12,2188-11-22,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Self Pay,HAIT,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,78552 51881 481 4239 9982 19889 78551 2859 197...,197 785 423 584 038 518 998 285 481 198 995,42 157 238 131 97 2 249 59 122
4,251,M,2090-03-15,117937,2110-07-27,2110-07-29,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,OTHER,NaN,UNKNOWN/NOT SPECIFIED,E8889 9100 85300 30500 9130,305 913 853 910 E888,660 2603 233 239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,44089,M,2026-05-25,165748,2111-09-30,2111-10-03,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,GREEK ORTHODOX,MARRIED,WHITE,4111 45829 71590 41401 60000 4019 30000 496,411 715 300 458 414 401 600 496,203 98 238 127 651 164 101
58925,44115,F,2124-07-27,163623,2161-07-15,2161-07-19,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,7840 78820 53081 3484,784 788 530 348,163 84 95 138
58926,44123,F,2049-11-26,116395,2135-01-06,2135-01-12,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,SEPARATED,WHITE,5845 5570 V667 78959 5853 496 44422 73679 4402...,286 E878 440 736 496 403 427 584 998 444 424 7...,106 259 157 53 127 116 208 114 55 96 2616 151 ...
58927,44126,F,2076-07-25,183530,2129-01-03,2129-01-11,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,7213 2851 4019 72403 75619 75614,285 756 724 401 721,98 60 217 205


In [27]:
data.SUBJECT_ID = data.SUBJECT_ID.apply(lambda x: str(x))
data.HADM_ID = data.HADM_ID.apply(lambda x: str(x))

In [28]:
data = data.sort_values(['SUBJECT_ID', 'ADMITTIME'], ascending=True).reset_index(drop=True)
data

,SUBJECT_ID,GENDER,DOB,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE
0,10,F,2103-06-28,184167,2103-06-28,2103-07-06,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,NaN,UNOBTAINABLE,NaN,BLACK/AFRICAN AMERICAN,V290 76515 76525 V3000 7742,765 V29 774 V30,222 218 219 256
1,100,F,2085-08-31,153952,2157-08-10,2157-08-18,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,4260 4411 4241 42731 99602,427 426 424 996 441,106 96 237 115 105
2,1000,M,2074-05-06,143040,2144-01-19,2144-02-25,URGENT,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,5789 2773 4280 2851 0380 40391 78559 20300 5185,203 785 277 578 038 518 285 428 403,58 60 131 99 108 153 2 40 249
3,10000,M,2136-12-12,187813,2186-08-10,2186-08-31,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,Self Pay,SPAN,CATHOLIC,NaN,HISPANIC OR LATINO,5845 28521 2766 25000 5997 5723 5712 0414 5770...,456 577 285 289 286 599 250 403 571 303 584 99...,163 157 55 660 238 152 99 64 49 134 3 211 151 ...
4,10001,M,2107-03-18,118420,2107-03-18,2107-03-20,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Government,NaN,BUDDHIST,NaN,ASIAN,V290 V502 V3000 V053,V29 V50 V05 V30,218 10 256 224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,9999,M,2091-02-27,117856,2134-12-26,2134-12-28,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicaid,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,79902 30301 2639 5711 2761 42741,799 427 571 303 263 276,55 660 52 107 244
58925,99991,M,2137-04-07,151118,2184-12-24,2185-01-05,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,71947 5570 40291 5538 5644 V0254 E8788 4280 25...,553 719 V02 584 562 038 250 E878 557 789 788 4...,163 157 114 55 145 238 99 49 108 2616 204 2 15...
58926,99992,F,2078-10-17,197084,2144-07-25,2144-07-28,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,5699 3004 53081 5772 2851 56881 4019 72992 579...,272 569 300 579 530 577 584 729 285 999 401 568,60 157 53 238 152 98 211 138 657 155
58927,99995,F,2058-05-29,137810,2147-02-08,2147-02-11,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,E8798 4400 99812 4241 9961 V5861 4280 99811 25...,272 V10 998 285 250 V15 E879 414 428 424 996 V...,60 114 53 238 24 49 96 108 2616 237 115 257 66...


## exclude patients < 18 yrs old

In [29]:
data['AGE'] = data.apply(lambda x: (x['ADMITTIME'] - x['DOB']).days // 365.25, axis=1)
data['AGE'] = data['AGE'].apply(lambda x: 89 if x > 89 else x)
data

,SUBJECT_ID,GENDER,DOB,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
0,10,F,2103-06-28,184167,2103-06-28,2103-07-06,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,NaN,UNOBTAINABLE,NaN,BLACK/AFRICAN AMERICAN,V290 76515 76525 V3000 7742,765 V29 774 V30,222 218 219 256,0.0
1,100,F,2085-08-31,153952,2157-08-10,2157-08-18,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,4260 4411 4241 42731 99602,427 426 424 996 441,106 96 237 115 105,71.0
2,1000,M,2074-05-06,143040,2144-01-19,2144-02-25,URGENT,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,5789 2773 4280 2851 0380 40391 78559 20300 5185,203 785 277 578 038 518 285 428 403,58 60 131 99 108 153 2 40 249,69.0
3,10000,M,2136-12-12,187813,2186-08-10,2186-08-31,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,Self Pay,SPAN,CATHOLIC,NaN,HISPANIC OR LATINO,5845 28521 2766 25000 5997 5723 5712 0414 5770...,456 577 285 289 286 599 250 403 571 303 584 99...,163 157 55 660 238 152 99 64 49 134 3 211 151 ...,49.0
4,10001,M,2107-03-18,118420,2107-03-18,2107-03-20,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Government,NaN,BUDDHIST,NaN,ASIAN,V290 V502 V3000 V053,V29 V50 V05 V30,218 10 256 224,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,9999,M,2091-02-27,117856,2134-12-26,2134-12-28,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicaid,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,79902 30301 2639 5711 2761 42741,799 427 571 303 263 276,55 660 52 107 244,43.0
58925,99991,M,2137-04-07,151118,2184-12-24,2185-01-05,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,71947 5570 40291 5538 5644 V0254 E8788 4280 25...,553 719 V02 584 562 038 250 E878 557 789 788 4...,163 157 114 55 145 238 99 49 108 2616 204 2 15...,47.0
58926,99992,F,2078-10-17,197084,2144-07-25,2144-07-28,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,5699 3004 53081 5772 2851 56881 4019 72992 579...,272 569 300 579 530 577 584 729 285 999 401 568,60 157 53 238 152 98 211 138 657 155,65.0
58927,99995,F,2058-05-29,137810,2147-02-08,2147-02-11,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,E8798 4400 99812 4241 9961 V5861 4280 99811 25...,272 V10 998 285 250 V15 E879 414 428 424 996 V...,60 114 53 238 24 49 96 108 2616 237 115 257 66...,88.0


In [30]:
data = data[data['AGE'] >= 18].reset_index(drop=True)
data = data.drop(columns='DOB')
data

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
0,100,F,153952,2157-08-10,2157-08-18,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,4260 4411 4241 42731 99602,427 426 424 996 441,106 96 237 115 105,71.0
1,1000,M,143040,2144-01-19,2144-02-25,URGENT,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,5789 2773 4280 2851 0380 40391 78559 20300 5185,203 785 277 578 038 518 285 428 403,58 60 131 99 108 153 2 40 249,69.0
2,10000,M,187813,2186-08-10,2186-08-31,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,Self Pay,SPAN,CATHOLIC,NaN,HISPANIC OR LATINO,5845 28521 2766 25000 5997 5723 5712 0414 5770...,456 577 285 289 286 599 250 403 571 303 584 99...,163 157 55 660 238 152 99 64 49 134 3 211 151 ...,49.0
3,10003,M,144039,2111-11-21,2111-12-02,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,OTHER,MARRIED,BLACK/AFRICAN AMERICAN,82525 4582 2899 99811 4019 82523 8602 E8798 80708,860 458 998 825 289 E879 401 807,238 234 98 64 2616 230 231,41.0
4,10004,M,164713,2182-11-02,2182-11-19,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,E8889 78039 7837 51881 4619 5225 3484 2720 803...,272 780 518 522 348 599 461 401 783 803 507 25...,58 159 233 53 98 131 49 126 95 83 663 136 2603...,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50714,9999,M,117856,2134-12-26,2134-12-28,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicaid,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,79902 30301 2639 5711 2761 42741,799 427 571 303 263 276,55 660 52 107 244,43.0
50715,99991,M,151118,2184-12-24,2185-01-05,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,71947 5570 40291 5538 5644 V0254 E8788 4280 25...,553 719 V02 584 562 038 250 E878 557 789 788 4...,163 157 114 55 145 238 99 49 108 2616 204 2 15...,47.0
50716,99992,F,197084,2144-07-25,2144-07-28,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,5699 3004 53081 5772 2851 56881 4019 72992 579...,272 569 300 579 530 577 584 729 285 999 401 568,60 157 53 238 152 98 211 138 657 155,65.0
50717,99995,F,137810,2147-02-08,2147-02-11,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,E8798 4400 99812 4241 9961 V5861 4280 99811 25...,272 V10 998 285 250 V15 E879 414 428 424 996 V...,60 114 53 238 24 49 96 108 2616 237 115 257 66...,88.0


## save

In [31]:
create_directory(os.path.join(data_path, 'mimic3/processed'))
data.to_csv(os.path.join(data_path, 'mimic3/processed/data.csv'), index=False)